# Impyte Documentation
This is a first practicle attempt to clarify the usage of `impyte`. It's a collection of easy applicable and reproducible examples that you could use to simplyify your data processing workflow.

## Importing and generating sample data
In order to show some of the features of the library, we'll be using Kaggle's HR data that can be found [here](https://www.kaggle.com/ludobenistant/hr-analytics).

In [2]:
# import library and data set
import impyte
import pandas as pd
from tools.data_prep import remove_random

In [3]:
data = pd.read_csv('data/hr_test.csv')

In [4]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


## Add random missing values
In order to show some of the pattern visualization and imputation methods, we need incomplete data sets. To achieve this, we'll be using a helper function that deletes values at random. 25 % of all values from each category will be deleted.

In [5]:
data = remove_random(data, .25, randomize_index=False)

In [6]:
data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.53,3.0,167.0,3.0,0.0,0.0,0.0,hr,low
1,NaN,0.49,3.0,199.0,10.0,0.0,0.0,0.0,technical,medium
2,NaN,0.46,2.0,128.0,3.0,0.0,1.0,0.0,accounting,low
3,NaN,0.96,4.0,154.0,4.0,0.0,0.0,0.0,support,medium
4,NaN,0.84,6.0,246.0,4.0,0.0,1.0,0.0,hr,medium


In [7]:
data.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000,14625.000000
mean,0.612897,0.716057,3.804991,200.968479,3.498735,0.144137,0.238222,0.021675
std,0.248577,0.171272,1.233198,49.980264,1.460087,0.351240,0.426010,0.145626
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [8]:
len(data)

14999

In [9]:
reload(impyte)

<module 'impyte' from 'impyte.pyc'>

## Testing rudimentary features
Below is a first trial run of some of the features implemented by impyter and its helper classes.

## `NanChecker`
Functionality testing of `NanChecker` class.

In [10]:
nan_checker = impyte.NanChecker()

#### `NanChecker.is_nan(data, nan_vals=None, recursive=True)`
Detect missing values (NaN in numeric arrays, empty strings in string arrays). NaN detection feature also enables NaN-value ingestion as parameter values.

In [11]:
# [True, False, False]
print nan_checker.is_nan(["", 'None', 'NaN'])

# [True, True, True]
print nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

[True, False, False]
[True, True, True]


In [12]:
# Recursive nan detection
# [True, True, False, [False, True, True]]
print nan_checker.is_nan(["", None, 'NaN', ["List Value 1", '', None]])

[True, True, False, [False, True, True]]


In [13]:
# Values can be declared as nan-values
# [True, False, False, True]
nan_checker.is_nan(['NaN', 'Empty', 'None', 'N/A'], nan_vals=['NaN', 'N/A'])

[True, False, False, True]

## `Pattern`
Functionality testing of `Pattern` class. The `Pattern` class stores different patterns and data summaries regarding NaN values. 

In [14]:
reload(impyte)
pattern_log = impyte.Pattern()

#### `Pattern._check_complete_row(row)`
Determines whether a row consists out of only 1s. Method for NaN summary creation

In [30]:
pd.DataFrame([1, 1, 1]).apply(pattern_log._check_complete_row)
#pattern_log._check_complete_row()

0   -1
dtype: int64

#### `Pattern._compute_pattern(data, nan_values="", verbose=False)`
Function that checks for missing values and prints out a quick table of a summary of missing values. Includes pattern overview and counts of missing values by column. Computes pattern and returns its pattern structure plus the count of data points for each of these patterns. To get a nice digestible table add `["table"]` selector to output.

In [16]:
pattern_dict = pattern_log._compute_pattern(data)
indices, table = pattern_dict["indices"], pattern_dict["table"]

In [17]:
table

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


In [18]:
indices[0][:10] # first 10 indices of pattern 0

[3740, 3741, 3742, 3743, 3744, 3745, 3746, 3747, 3748, 3749]

In [19]:
pattern_nr = 0
print "Pattern {} has {:,} rows.".format(pattern_nr, len(indices[0])) 

Pattern 0 has 11,259 rows.


#### `Pattern._get_discrete_and_continuous(tmpdata, unique_instances)`
Returns the column names of discrete and continuous variables. Column names are stored in lists for easy selection. Those lists are stored in one dictionary object. All continuous column names can be accessed through `['continuous']` all discrete variables through `['discrete']`.

In [15]:
pattern_log._get_discrete_and_continuous(data, unique_instances=5)

{'continuous': ['satisfaction_level',
  'last_evaluation',
  'number_project',
  'average_montly_hours',
  'time_spend_company'],
 'discrete': ['Work_accident',
  'left',
  'promotion_last_5years',
  'sales',
  'salary']}

#### `Pattern.get_pattern()`
Returns NaN-patterns based on primary computation or initiates new computation of NaN-patterns. Uses `_compute_patterns()` to visualize patterns if not yet computed. Otherwise returns stored information regarding patterns.

In [20]:
pattern_log.get_pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


#### `Pattern.row_nan_pattern(row)`
This is a core piece to determining which patterns are available. Other methods work with the output of `row_nan_pattern()` to determine unique pattern structures and count these before turning them into readable tables.

In [21]:
# (1, 1, 'NaN', 'NaN')
print pattern_log.row_nan_pattern(['Value 1', 'Value 2', '', None])

(1, 1, 'NaN', 'NaN')


#### `Pattern.get_pattern_indices(pattern_no)`

In [22]:
pattern_log.get_pattern_indices(0)[:10] # get first 10 indices of pattern 0

[3740, 3741, 3742, 3743, 3744, 3745, 3746, 3747, 3748, 3749]

#### `Pattern.get_continuous()`
Returns list with names of all continuous variables.

In [23]:
pattern_log.get_continuous()

['satisfaction_level', 'last_evaluation', 'average_montly_hours']

#### `Pattern.get_discrete()`
Returns list with names of all discrete variables.

In [24]:
pattern_log.get_discrete()

['number_project',
 'time_spend_company',
 'Work_accident',
 'left',
 'promotion_last_5years',
 'sales',
 'salary']

## `Impyter`

In [25]:
reload(impyte)
imp = impyte.Impyter() # instantiate impyte class

### Load data into imputer

#### `Impyter.load_data()`
Requires a pandas DataFrame to load. Otherwise, the input is being transformed into a DataFrame. While loading the data is being copied into the object, to stay clear of consistency issues with the original data set.

In [26]:
imp.load_data(data)

As an alternative a DataFrame can be handed over while instantiating the Impyter object.

In [27]:
imp = impyte.Impyter(data)

#### `Impyter.get_data()`
Returns the loaded data for quick reference.

In [28]:
imp.get_data().head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.94,3.0,162.0,3.0,1.0,0.0,0.0,sales,medium
1,NaN,0.90,5.0,254.0,6.0,0.0,1.0,0.0,support,low
2,NaN,0.94,4.0,144.0,3.0,0.0,0.0,0.0,support,low
3,NaN,0.92,3.0,199.0,3.0,1.0,0.0,0.0,management,medium
4,NaN,0.53,2.0,154.0,3.0,0.0,1.0,0.0,IT,low


#### `Impyter.pattern()`
Leverages `Pattern._compute_pattern()` and `Pattern.get_pattern()` methods to compute and return an overview of all existant NaN patterns in the data set. The overview shows a `NaN` in the column where a data point was missing and `1` for all complete slots. On the right hand side is a count variable to indicate how often that pattern was found. The patterns are always sorted by count and it is not given, that pattern 0 is always the pattern with only complete cases.

In [29]:
reload(impyte)
imp = impyte.Impyter(data) # instantiate impyte class
nan_checker = impyte.NanChecker()
pattern_log = impyte.Pattern()
imp.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


#### `Impyter.drop_pattern()`
Drops pattern from data set.

In [30]:
imp.drop_pattern(4).head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.94,3.0,162.0,3.0,1.0,0.0,0.0,sales,medium
1,NaN,0.90,5.0,254.0,6.0,0.0,1.0,0.0,support,low
2,NaN,0.94,4.0,144.0,3.0,0.0,0.0,0.0,support,low
3,NaN,0.92,3.0,199.0,3.0,1.0,0.0,0.0,management,medium
4,NaN,0.53,2.0,154.0,3.0,0.0,1.0,0.0,IT,low


In [31]:
imp.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
4,1,1,1,1,1,1,1,1,1,NaN,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374


If inplace flag set to True, the changes happen in the data set that's being stored in the Impyte object. Otherwise, a copy without the dropped pattern will be returned and the stored data set stays intact.

In [32]:
_ = imp.drop_pattern(4, inplace=True)

In [33]:
imp.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,11259
1,1,1,1,1,1,1,1,1,NaN,1,374
2,1,1,1,NaN,1,1,1,1,1,1,374
3,1,1,NaN,1,1,1,1,1,1,1,374
5,1,1,1,1,1,1,1,NaN,1,1,374
6,1,1,1,1,NaN,1,1,1,1,1,374
7,NaN,1,1,1,1,1,1,1,1,1,374
8,1,NaN,1,1,1,1,1,1,1,1,374
9,1,1,1,1,1,NaN,1,1,1,1,374
10,1,1,1,1,1,1,NaN,1,1,1,374


#### `Impyter.get_missing_summary()`
Returns table with information on missing values per column and its percentage.

In [34]:
imp.get_missing_summary()

,Complete,Missing,Percentage
salary,14625,0,0.00 %
satisfaction_level,14251,374,2.56 %
last_evaluation,14251,374,2.56 %
number_project,14251,374,2.56 %
average_montly_hours,14251,374,2.56 %
time_spend_company,14251,374,2.56 %
Work_accident,14251,374,2.56 %
left,14251,374,2.56 %
promotion_last_5years,14251,374,2.56 %
sales,14251,374,2.56 %


Setting the importance filter flag to `True`  shows only columns that have some missing values. This is helpful for data sets with a large amount of variables and only few nan-values.

In [35]:
for pattern_no in range(1,4): #  drop patterns 1 to 3
    imp.drop_pattern(pattern_no, inplace=True)

In [36]:
imp.get_missing_summary(importance_filter=True)

,Complete,Missing,Percentage
satisfaction_level,13129,374,2.77 %
last_evaluation,13129,374,2.77 %
time_spend_company,13129,374,2.77 %
Work_accident,13129,374,2.77 %
left,13129,374,2.77 %
promotion_last_5years,13129,374,2.77 %


#### `Impyte.one_hot_encode()`
Relies on `pandas.get_dummies()` method to transform categorical values into one-hot-encoded values.

In [37]:
reload(impyte)
imputer = impyte.Impyter()
imputer.load_data(data)
_ = imputer.pattern()

In [38]:
ohe_data = imputer.one_hot_encode(data)
ohe_data.head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales_ohe_IT,sales_ohe_RandD,...,sales_ohe_hr,sales_ohe_management,sales_ohe_marketing,sales_ohe_product_mng,sales_ohe_sales,sales_ohe_support,sales_ohe_technical,salary_ohe_high,salary_ohe_low,salary_ohe_medium
0,NaN,0.94,3.0,162.0,3.0,1.0,0.0,0.0,0,0,...,0,0,0,0,1,0,0,0,0,1
1,NaN,0.90,5.0,254.0,6.0,0.0,1.0,0.0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,NaN,0.94,4.0,144.0,3.0,0.0,0.0,0.0,0,0,...,0,0,0,0,0,1,0,0,1,0
3,NaN,0.92,3.0,199.0,3.0,1.0,0.0,0.0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,NaN,0.53,2.0,154.0,3.0,0.0,1.0,0.0,1,0,...,0,0,0,0,0,0,0,0,1,0


#### `Impyte.one_hot_decode()`
The inversion method to `Impyte.one_hot_encode()`. Transforms one-hot-encoded columns back to categorical values.

In [39]:
imputer.one_hot_decode(ohe_data).head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,NaN,0.94,3.0,162.0,3.0,1.0,0.0,0.0,sales,medium
1,NaN,0.90,5.0,254.0,6.0,0.0,1.0,0.0,support,low
2,NaN,0.94,4.0,144.0,3.0,0.0,0.0,0.0,support,low
3,NaN,0.92,3.0,199.0,3.0,1.0,0.0,0.0,management,medium
4,NaN,0.53,2.0,154.0,3.0,0.0,1.0,0.0,IT,low


#### `Impyter.impute()`
Impute is the core method of impyte. The method works out of the box and uses Random Forest estimators per default to impute missing values. It automatically performs cross-validation to showcase the potential accuracy of the imputation. Scoring that is being used is f1_macro score for classifiers (supporting binary and multi-class) and r2 for regression models.

In [40]:
reload(impyte)
imputer = impyte.Impyter()
imputer.load_data(data)
_ = imputer.pattern()
complete_df = imputer.impute(classifier='rf')

Label: sales 	 Fitting RandomForestClassifier 	 Avg score: 0.336 (f1_macro)
Label: average_montly_hours 	 Fitting RandomForestRegressor 	 Avg score: 0.327 (r2)
Label: number_project 	 Fitting RandomForestClassifier 	 Avg score: 0.574 (f1_macro)
Label: salary 	 Fitting RandomForestClassifier 	 Avg score: 0.507 (f1_macro)
Label: promotion_last_5years 	 Fitting RandomForestClassifier 	 Avg score: 0.736 (f1_macro)
Label: time_spend_company 	 Fitting RandomForestClassifier 	 Avg score: 0.575 (f1_macro)
Label: satisfaction_level 	 Fitting RandomForestRegressor 	 Avg score: 0.459 (r2)
Label: last_evaluation 	 Fitting RandomForestRegressor 	 Avg score: 0.268 (r2)
Label: Work_accident 	 Fitting RandomForestClassifier 	 Avg score: 0.592 (f1_macro)
Label: left 	 Fitting RandomForestClassifier 	 Avg score: 0.979 (f1_macro)


### Investigate completed data set

In [41]:
imp = impyte.Impyter(complete_df)

In [42]:
imp.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,14999


#### `Impyter.get_result()`
Returns the results data set, once impute was performed.

In [43]:
imputer.get_result().head()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
0,0.781,0.94,3.0,162.0,3.0,1.0,0.0,0.0,sales,medium
1,0.858,0.90,5.0,254.0,6.0,0.0,1.0,0.0,support,low
2,0.620,0.94,4.0,144.0,3.0,0.0,0.0,0.0,support,low
3,0.794,0.92,3.0,199.0,3.0,1.0,0.0,0.0,management,medium
4,0.425,0.53,2.0,154.0,3.0,0.0,1.0,0.0,IT,low


### Comparison of complete indices methods

In [44]:
%timeit len(imputer.pattern_log.get_complete_indices())

100 loops, best of 3: 906 µs per loop


In [45]:
%timeit len(imputer.get_complete_old())

100 loops, best of 3: 6.71 ms per loop


In [46]:
_ = imputer.drop_pattern(7, inplace=True)

In [47]:
data.iloc[:, imputer.pattern_log.store_tuple_columns[(1, 1, 1, 1, 1, 1, 1, 1, 1, 'NaN')]].columns

Index([u'salary'], dtype='object')

In [48]:
data.tail()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
14994,0.60,0.60,3.0,258.0,3.0,0.0,0.0,0.0,sales,low
14995,0.77,0.54,4.0,139.0,3.0,1.0,0.0,0.0,IT,low
14996,0.73,0.87,4.0,155.0,3.0,1.0,0.0,0.0,technical,high
14997,0.76,0.72,3.0,242.0,5.0,1.0,0.0,0.0,technical,medium
14998,0.66,0.83,4.0,266.0,3.0,1.0,0.0,0.0,technical,medium


### Accessing final models

In [49]:
mdl = imputer.get_model(2) # returns model for pattern 2

In [50]:
mdl.get_feature_name()

'average_montly_hours'

In [51]:
mdl.get_model()

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

### Scaling for preprocessing

In [52]:
from sklearn.preprocessing import StandardScaler

In [53]:
scaler = StandardScaler()

In [54]:
test = data.dropna()

In [55]:
len(test[test.apply(nan_checker.is_nan)])

11259

In [56]:
test.describe()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years
count,11259.000000,11259.000000,11259.000000,11259.000000,11259.000000,11259.000000,11259.000000,11259.000000
mean,0.612411,0.715967,3.797140,200.820055,3.492495,0.141398,0.239453,0.020695
std,0.247983,0.171836,1.234197,50.064894,1.457094,0.348447,0.426769,0.142366
min,0.090000,0.360000,2.000000,96.000000,2.000000,0.000000,0.000000,0.000000
25%,0.440000,0.560000,3.000000,156.000000,3.000000,0.000000,0.000000,0.000000
50%,0.640000,0.720000,4.000000,200.000000,3.000000,0.000000,0.000000,0.000000
75%,0.820000,0.870000,5.000000,245.000000,4.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,7.000000,310.000000,10.000000,1.000000,1.000000,1.000000


In [57]:
test_scaled = scaler.fit_transform(test[test.corr().columns])

In [58]:
test[:5]

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary
3740,0.74,0.62,4.0,236.0,4.0,0.0,0.0,0.0,sales,low
3741,0.43,0.48,2.0,147.0,3.0,0.0,1.0,0.0,IT,medium
3742,0.45,0.53,2.0,149.0,3.0,0.0,1.0,0.0,support,low
3743,0.80,0.65,3.0,264.0,4.0,0.0,0.0,0.0,management,high
3744,0.97,0.38,5.0,211.0,7.0,1.0,0.0,0.0,sales,medium


In [59]:
pd.DataFrame(test_scaled[:5])

,0,1,2,3,4,5,6,7
0,0.514532,-0.558502,0.164373,0.702718,0.348315,-0.405813,-0.561109,-0.145368
1,-0.735609,-1.373267,-1.456185,-1.075054,-0.338013,-0.405813,1.782186,-0.145368
2,-0.654955,-1.082279,-1.456185,-1.035104,-0.338013,-0.405813,1.782186,-0.145368
3,0.756494,-0.383910,-0.645906,1.262017,0.348315,-0.405813,-0.561109,-0.145368
4,1.442055,-1.955241,0.974652,0.203344,2.407299,2.464191,-0.561109,-0.145368


In [60]:
test_rescaled = scaler.inverse_transform(test_scaled)

In [61]:
pd.DataFrame(test_rescaled[:5])

,0,1,2,3,4,5,6,7
0,0.74,0.62,4.0,236.0,4.0,0.0,2.775558e-17,0.0
1,0.43,0.48,2.0,147.0,3.0,0.0,1.000000e+00,0.0
2,0.45,0.53,2.0,149.0,3.0,0.0,1.000000e+00,0.0
3,0.80,0.65,3.0,264.0,4.0,0.0,2.775558e-17,0.0
4,0.97,0.38,5.0,211.0,7.0,1.0,2.775558e-17,0.0


In [66]:
reload(impyte)
testdf = pd.read_csv('data/masterdf_201710230_andirs.csv', low_memory=False)
imp = impyte.Impyter()
_ = imp.load_data(testdf)
imp.pattern()

,Unnamed: 0,Incident Date,EAS,Incident_Year,Incident_Cat,Incident_Dummy,Neighborhood,Location_y,Address,Building_Cat,...,count all complaints not corrected,count potential fire control not corrected,count fire emergency safety,count potential fire cause,count fire emergency safety not corrected,count potential fire cause not corrected,next_fire_dpt_address,next_fire_dpt_distance,next_fire_dpt_latlong,Count
0,1,NaN,1,NaN,NaN,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,170517
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,24767
2,1,1,1,1,NaN,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,24


In [63]:
imp.get_missing_summary()

,Complete,Missing,Percentage
Unnamed: 0,195308,0,0.00 %
next_fire_dpt_address,195308,0,0.00 %
count potential fire cause not corrected,195308,0,0.00 %
count fire emergency safety not corrected,195308,0,0.00 %
count potential fire cause,195308,0,0.00 %
count fire emergency safety,195308,0,0.00 %
count potential fire control not corrected,195308,0,0.00 %
count all complaints not corrected,195308,0,0.00 %
count all complaints,195308,0,0.00 %
count potential fire control,195308,0,0.00 %


In [64]:
imp.get_missing_summary(importance_filter=True)

,Complete,Missing,Percentage
Incident_Year,24791,170517,87.31 %
Incident Date,24791,170517,87.31 %
Incident_Cat,24767,170541,87.32 %


### Timing some of the functions
Below is a piece of information on efficiency of the functions. This doesn't reflect complexity of the functions, but it will show an runtime estimate for your local machine.

In [65]:
%timeit nan_checker.is_nan(["", 'None', 'NaN'])
%timeit nan_checker.is_nan(["", 'None', 'NaN'], nan_vals=['', None, 'None', 'NaN'])

The slowest run took 18.02 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 5.11 µs per loop
The slowest run took 4.28 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 3: 5.12 µs per loop


In [ ]:
multi_data = data.copy()
import numpy as np
for i in range(0, 100):
    multi_data.at[i, "last_evaluation"] = np.nan
reload(impyte)
imp = impyte.Impyter(multi_data)
_ = imp.pattern()
imp.pattern_log.get_multi_nan_pattern_nos()
imp.pattern_log.get_column_name(11)
res = imp.impute(multi_nans=True)

In [139]:
imp2 = impyte.Impyter(res)
imp2.pattern()

,satisfaction_level,last_evaluation,number_project,average_montly_hours,time_spend_company,Work_accident,left,promotion_last_5years,sales,salary,Count
0,1,1,1,1,1,1,1,1,1,1,14899
1,NaN,NaN,1,1,1,1,1,1,1,1,100
